In [1]:
!uv pip install transformers

Using Python 3.12.9 environment at: /workspaces/fundamentals-of-ai-engineering-principles-and-practical-applications-6026542/.venv
Audited 1 package in 119ms


In [2]:
# pip install transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
checkpoint = "HuggingFaceTB/SmolLM-135M"
device = "cpu"  # for GPU usage or "cpu" for CPU usage
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# for multiple GPUs install accelerate and do `model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")`
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)
inputs = tokenizer.encode("def print_hello_world():",
                          return_tensors="pt").to(device)
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))

tokenizer_config.json:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


def print_hello_world():
    print("Hello World!")

# print_hello_world()

# print_


In [ ]:
# Simple Local LLM Demo (Fixed Version)
# For LinkedIn Learning course chapter on running local LLMs

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


def run_tiny_llm():
    print("Loading TinyLlama-1.1B model - one of the smallest practical LLMs...")

    # Load the model and tokenizer
    # TinyLlama-1.1B is about 1.1B parameters - very small for an LLM
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Load model with minimal settings (no quantization)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if torch.cuda.is_available() else None,
        device_map="auto" if torch.cuda.is_available() else None
    )

    # Move to device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    if device == "cpu":
        model = model.to(device)

    print(f"Model loaded successfully!")
    print(f"Running on: {next(model.parameters()).device}")

    # Function to generate text
    def generate_response(prompt, max_length=100):
        # Format the prompt according to the model's expected format
        formatted_prompt = f"<|user|>\n{prompt}\n<|assistant|>\n"

        # Tokenize the prompt
        inputs = tokenizer(
            formatted_prompt, return_tensors="pt").to(model.device)

        # Generate
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_length,
                temperature=0.7,
                top_p=0.9,
                do_sample=True
            )

        # Decode and return
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract just the assistant's response
        response = response.split("<|assistant|>")[-1].strip()
        return response

    # Demo the model with a few examples
    examples = [
        "What is machine learning?",
        "Write a short poem about artificial intelligence.",
        "Explain how to make a cup of coffee in 3 steps."
    ]

    print("\n" + "="*50)
    print("DEMONSTRATION OF TINY LOCAL LLM CAPABILITIES")
    print("="*50)

    for i, example in enumerate(examples, 1):
        print(f"\nExample {i}: \"{example}\"")
        print("-" * 40)
        response = generate_response(example)
        print(f"Response: {response}")
        print("-" * 40)

    # Interactive mode
    print("\n" + "="*50)
    print("INTERACTIVE MODE - Type 'exit' to quit")
    print("="*50)

    while True:
        user_input = input("\nEnter your prompt: ")
        if user_input.lower() == 'exit':
            break

        response = generate_response(user_input)
        print(f"\nResponse: {response}")

    print("\nDemo completed!")


if __name__ == "__main__":
    # Check if CUDA is available
    if torch.cuda.is_available():
        print(f"CUDA available: {torch.cuda.get_device_name(0)}")
    else:
        print("CUDA not available. Running on CPU which will be slow.")
        print(
            "For better performance, consider using Google Colab or a machine with a GPU.")

    run_tiny_llm()

CUDA not available. Running on CPU which will be slow.
For better performance, consider using Google Colab or a machine with a GPU.
Loading TinyLlama-1.1B model - one of the smallest practical LLMs...


model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model loaded successfully!
Running on: cpu

DEMONSTRATION OF TINY LOCAL LLM CAPABILITIES

Example 1: "What is machine learning?"
----------------------------------------
Response: Machine learning is a field of artificial intelligence that involves the use of algorithms to learn and improve from data. In simple terms, machine learning is the process of using data to teach a computer to perform tasks that it was not explicitly designed to perform. It involves developing algorithms that can learn from data without being explicitly programmed, and then using these algorithms to make predictions, recommendations, or decisions.
Some examples of machine learning algorithms include:
1. Decision trees: This algorithm is used
----------------------------------------

Example 2: "Write a short poem about artificial intelligence."
----------------------------------------
Response: A world of wonder and new discoveries,
A future of infinite possibilities,
Where machines can think and reason like h

In [2]:
# Fixed Minimal LLM Demo - CPU Only
# For LinkedIn Learning course on local LLMs

from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the smallest practical model (DistilGPT2 - 15M parameters)
model_name = "distilbert/distilgpt2"
print(f"Loading {model_name}...")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Fix: Set proper padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("Model loaded successfully!")

# Fixed function to generate text with proper attention mask


def generate_text(prompt, max_length=50):
    # Encode the input with attention mask
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        return_attention_mask=True
    )

    # Generate with proper parameters
    output = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=max_length,
        do_sample=True,
        temperature=0.8,
        pad_token_id=tokenizer.pad_token_id
    )

    # Decode and return
    return tokenizer.decode(output[0], skip_special_tokens=True)


# Example prompts
examples = [
    "Artificial intelligence is",
    "The future of technology",
    "Learning to code"
]

print("\nDEMONSTRATION OF TINY LLM")

# Run examples
for example in examples:
    print(f"\nPrompt: \"{example}\"")
    response = generate_text(example)
    print(f"Generated: {response}")

Loading distilbert/distilgpt2...
Model loaded successfully!

DEMONSTRATION OF TINY LLM

Prompt: "Artificial intelligence is"
Generated: Artificial intelligence is one of the most powerful tools in recent history. It can detect and classify information in the form of pictures, videos, pictures, pictures, images or any other information. It can also pick out patterns of behavior and predict what to

Prompt: "The future of technology"
Generated: The future of technology is uncertain, as well as the potential for other forms of information disruption.
































Prompt: "Learning to code"
Generated: Learning to code and make life a more livable environment.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model_name = "distilbert/distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set padding token
tokenizer.pad_token = tokenizer.eos_token

# Simple generation function


def generate_text(prompt, max_length=50):
    inputs = tokenizer(prompt, return_tensors="pt", return_attention_mask=True)
    output = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=max_length,
        do_sample=True,
        temperature=0.8,
        pad_token_id=tokenizer.pad_token_id
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)


# Example usage
prompt = "Artificial intelligence is"
response = generate_text(prompt)
print(f"Prompt: \"{prompt}\"")
print(f"Generated: {response}")

Prompt: "Artificial intelligence is"
Generated: Artificial intelligence is a topic that has been hotly debated. But, as such, we're not the only ones that want to learn.


In [4]:
# Import only what's needed
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

# Set the directory where you want to save the model
save_directory = "./saved_model"  # Change this to your preferred path

# Create the directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

# Load model and tokenizer
model_name = "distilbert/distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set padding token
tokenizer.pad_token = tokenizer.eos_token

# Save the model and tokenizer to the specified directory
print(f"Saving model to {save_directory}...")
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
print("Model and tokenizer saved successfully!")

# Now we can load from local directory instead of downloading again
print("Loading model from local directory...")
local_model = AutoModelForCausalLM.from_pretrained(save_directory)
local_tokenizer = AutoTokenizer.from_pretrained(save_directory)
local_tokenizer.pad_token = local_tokenizer.eos_token
print("Model loaded from local directory!")

# Simple generation function using the local model


def generate_text(prompt, max_length=50):
    inputs = local_tokenizer(prompt, return_tensors="pt",
                             return_attention_mask=True)
    output = local_model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=max_length,
        do_sample=True,
        temperature=0.8,
        pad_token_id=local_tokenizer.pad_token_id
    )
    return local_tokenizer.decode(output[0], skip_special_tokens=True)


# Example usage
prompt = "Artificial intelligence is"
response = generate_text(prompt)
print(f"Prompt: \"{prompt}\"")
print(f"Generated: {response}")

Saving model to ./saved_model...
Model and tokenizer saved successfully!
Loading model from local directory...
Model loaded from local directory!
Prompt: "Artificial intelligence is"
Generated: Artificial intelligence is not merely a tool for solving problems but an important tool to help solve them. It is a powerful tool that can lead to meaningful and lasting results for both humans and people.


To learn about Artificial Intelligence, please visit


In [6]:
import torch
import transformers
import numpy as np

In [7]:
# Breaking down the inference process step by step

# Step 1: Load tokenizer and model
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "distilbert/distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Step 2: Prepare input text and convert to tokens
input_text = "Artificial intelligence will"
print(f"Original text: {input_text}")

# Tokenization: Convert text to token IDs
tokens = tokenizer.encode(input_text)
print(f"Token IDs: {tokens}")

# See the actual tokens
tokens_decoded = [tokenizer.decode([token]) for token in tokens]
print(f"Individual tokens: {tokens_decoded}")

# Step 3: Prepare inputs for the model (add batch dimension and attention mask)
inputs = tokenizer(input_text, return_tensors="pt", return_attention_mask=True)
print(f"Model input structure: {inputs.keys()}")
print(f"Input shape: {inputs.input_ids.shape}")

# Step 4: Generate text (the actual inference)
with torch.no_grad():  # Disable gradient calculation for inference
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=50,
        do_sample=True,
        temperature=0.8,
        pad_token_id=tokenizer.eos_token_id
    )

# Step 5: Decode the generated tokens back to text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Generated text: {generated_text}")

Original text: Artificial intelligence will
Token IDs: [8001, 9542, 4430, 481]
Individual tokens: ['Art', 'ificial', ' intelligence', ' will']
Model input structure: dict_keys(['input_ids', 'attention_mask'])
Input shape: torch.Size([1, 4])
Generated text: Artificial intelligence will be a significant step towards a future where companies are able to develop and manage AI. Artificial intelligence will be at a high level that we are already seeing.

















In [8]:
# Exploring tokenization in detail

# 1. Different tokenization approaches
input_text = "Artificial intelligence models like GPT-2 transform text into tokens."

# Show how the tokenizer breaks down text
tokens = tokenizer.tokenize(input_text)
print(f"Input text: {input_text}")
print(f"Tokenized: {tokens}")
print(f"Number of tokens: {len(tokens)}")

# 2. Special tokens and their purpose
print(f"\nSpecial tokens in this tokenizer:")
print(
    f"BOS (beginning of sequence) token: {tokenizer.bos_token}, ID: {tokenizer.bos_token_id}")
print(
    f"EOS (end of sequence) token: {tokenizer.eos_token}, ID: {tokenizer.eos_token_id}")
print(
    f"PAD (padding) token: {tokenizer.pad_token}, ID: {tokenizer.pad_token_id}")
print(
    f"UNK (unknown) token: {tokenizer.unk_token}, ID: {tokenizer.unk_token_id}")

# 3. Handling out-of-vocabulary words
unusual_words = ["cryptocurrencies", "transformers",
                 "COVID19", "😊", "anthropomorphization"]
print("\nTokenizing unusual or new words:")
for word in unusual_words:
    word_tokens = tokenizer.tokenize(word)
    print(f"'{word}' → {word_tokens}")

# 4. Demonstrate context window with token counting
long_text = "This is " + "very " * 50 + "long."
tokens = tokenizer.tokenize(long_text)
print(f"\nLong text token count: {len(tokens)}")
print(
    f"Maximum context length for this model: {model.config.max_position_embeddings}")

# 5. Show the effect of truncation and padding
texts = ["Short text", "This is a bit longer text with more content", "A" * 100]
print("\nHandling texts of different lengths:")

# Without padding/truncation
encoded = [tokenizer.encode(t) for t in texts]
for i, e in enumerate(encoded):
    print(f"Text {i+1} length: {len(e)} tokens")

# With padding and truncation
encoded_padded = tokenizer(
    texts, padding=True, truncation=True, max_length=20, return_tensors="pt")
print(f"\nAfter padding/truncation to length 20:")
print(f"Input shape: {encoded_padded.input_ids.shape}")
print(f"Attention mask: \n{encoded_padded.attention_mask}")

Input text: Artificial intelligence models like GPT-2 transform text into tokens.
Tokenized: ['Art', 'ificial', 'Ġintelligence', 'Ġmodels', 'Ġlike', 'ĠG', 'PT', '-', '2', 'Ġtransform', 'Ġtext', 'Ġinto', 'Ġtokens', '.']
Number of tokens: 14

Special tokens in this tokenizer:
BOS (beginning of sequence) token: <|endoftext|>, ID: 50256
EOS (end of sequence) token: <|endoftext|>, ID: 50256
PAD (padding) token: None, ID: None
UNK (unknown) token: <|endoftext|>, ID: 50256

Tokenizing unusual or new words:
'cryptocurrencies' → ['crypt', 'oc', 'urrencies']
'transformers' → ['transform', 'ers']
'COVID19' → ['CO', 'VID', '19']
'😊' → ['ðŁĺ', 'Ĭ']
'anthropomorphization' → ['anthrop', 'omorph', 'ization']

Long text token count: 54
Maximum context length for this model: 1024

Handling texts of different lengths:
Text 1 length: 2 tokens
Text 2 length: 9 tokens
Text 3 length: 13 tokens


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [9]:
# Exploring tokenization in detail

# 1. Load the tokenizer
from transformers import AutoTokenizer

model_name = "distilbert/distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Set padding token (required for batched operations)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 2. Different tokenization approaches
input_text = "Artificial intelligence models like GPT-2 transform text into tokens."

# Show how the tokenizer breaks down text
tokens = tokenizer.tokenize(input_text)
print(f"Input text: {input_text}")
print(f"Tokenized: {tokens}")
print(f"Number of tokens: {len(tokens)}")

# 3. Special tokens and their purpose
print(f"\nSpecial tokens in this tokenizer:")
print(
    f"BOS (beginning of sequence) token: {tokenizer.bos_token}, ID: {tokenizer.bos_token_id}")
print(
    f"EOS (end of sequence) token: {tokenizer.eos_token}, ID: {tokenizer.eos_token_id}")
print(
    f"PAD (padding) token: {tokenizer.pad_token}, ID: {tokenizer.pad_token_id}")
print(
    f"UNK (unknown) token: {tokenizer.unk_token}, ID: {tokenizer.unk_token_id}")

# 4. Handling out-of-vocabulary words
unusual_words = ["cryptocurrencies",
                 "transformers", "COVID19", "AI2023", "neural"]
print("\nTokenizing unusual or new words:")
for word in unusual_words:
    word_tokens = tokenizer.tokenize(word)
    print(f"'{word}' → {word_tokens}")

# 5. Demonstrate token IDs
print("\nConverting tokens to IDs:")
token_ids = tokenizer.encode(input_text)
print(f"Token IDs: {token_ids}")

# 6. Show the effect of truncation and padding
texts = ["Short text", "This is a bit longer text with more content", "A" * 100]
print("\nHandling texts of different lengths:")

# Without padding/truncation - just show lengths
encoded = [tokenizer.encode(t) for t in texts]
for i, e in enumerate(encoded):
    print(f"Text {i+1} length: {len(e)} tokens")

# With padding and truncation
encoded_padded = tokenizer(
    texts,
    padding=True,
    truncation=True,
    max_length=20,
    return_tensors="pt"
)
print(f"\nAfter padding/truncation to length 20:")
print(f"Input shape: {encoded_padded.input_ids.shape}")
print(f"Sample padded sequence: {encoded_padded.input_ids[0]}")
print(
    f"Attention mask (1=real token, 0=padding): \n{encoded_padded.attention_mask}")

# 7. Decoding back to text
original_text = "AI is changing the world"
print(f"\nOriginal text: {original_text}")

# Encode and then decode
encoded_text = tokenizer.encode(original_text)
decoded_text = tokenizer.decode(encoded_text)
print(f"Encoded and then decoded: {decoded_text}")

# 8. Subword tokenization example
print("\nSubword tokenization examples:")
examples = ["unhappiness", "multilingual", "pretrained"]
for example in examples:
    tokens = tokenizer.tokenize(example)
    print(f"'{example}' → {tokens}")

# 9. Context window demonstration with a longer text
some_paragraphs = " ".join(["This is paragraph " + str(i) +
                           "." + " It contains some text." * 5 for i in range(10)])
tokens = tokenizer.tokenize(some_paragraphs)
print(f"\nLong text example:")
print(f"Number of characters: {len(some_paragraphs)}")
print(f"Number of tokens: {len(tokens)}")
print(f"First 10 tokens: {tokens[:10]}")
print(f"Last 10 tokens: {tokens[-10:]}")

Input text: Artificial intelligence models like GPT-2 transform text into tokens.
Tokenized: ['Art', 'ificial', 'Ġintelligence', 'Ġmodels', 'Ġlike', 'ĠG', 'PT', '-', '2', 'Ġtransform', 'Ġtext', 'Ġinto', 'Ġtokens', '.']
Number of tokens: 14

Special tokens in this tokenizer:
BOS (beginning of sequence) token: <|endoftext|>, ID: 50256
EOS (end of sequence) token: <|endoftext|>, ID: 50256
PAD (padding) token: <|endoftext|>, ID: 50256
UNK (unknown) token: <|endoftext|>, ID: 50256

Tokenizing unusual or new words:
'cryptocurrencies' → ['crypt', 'oc', 'urrencies']
'transformers' → ['transform', 'ers']
'COVID19' → ['CO', 'VID', '19']
'AI2023' → ['AI', '20', '23']
'neural' → ['ne', 'ural']

Converting tokens to IDs:
Token IDs: [8001, 9542, 4430, 4981, 588, 402, 11571, 12, 17, 6121, 2420, 656, 16326, 13]

Handling texts of different lengths:
Text 1 length: 2 tokens
Text 2 length: 9 tokens
Text 3 length: 13 tokens

After padding/truncation to length 20:
Input shape: torch.Size([3, 13])
Sample pa